In [ ]:
from pathlib import Path
import pandas as pd

from graphrag.config.load_config import load_config

In [ ]:
PROJECT_DIRECTORY = "./handson"
graphrag_config = load_config(Path(PROJECT_DIRECTORY))

Model config based on fnllm is deprecated and will be removed in GraphRAG v3, please use chat or embedding instead to switch to LiteLLM config.
Model config based on fnllm is deprecated and will be removed in GraphRAG v3, please use chat or embedding instead to switch to LiteLLM config.


In [3]:
entities_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/entities.parquet")
communities_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/communities.parquet")
community_reports_df = pd.read_parquet(
    f"{PROJECT_DIRECTORY}/output/community_reports.parquet"
)
relationships_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/relationships.parquet")
documents_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/documents.parquet")
text_units_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/text_units.parquet")

In [ ]:
from graphrag.query.indexer_adapters import (
    read_indexer_entities,
    read_indexer_relationships,
)

entities = read_indexer_entities(entities_df, communities_df , None)

entities_dict = []
for e in entities:
    entity = {
        "id": e.title,
        "properties":{
            "id": e.id,
            "title": e.title,
            "type": e.type,
            "description": e.description,
            "community_ids": e.community_ids,
            "text_unit_ids": e.text_unit_ids,
            "rank": e.rank
            }
        }
    entities_dict.append(entity)

relationships = read_indexer_relationships(relationships_df)
relationships_dict = []
for r in relationships:
    relationship = {
        "start": r.source,
        "end": r.target,
        "properties": {
            "id": r.id,
            "source": r.source,
            "target": r.target,
            "description": r.description,
            "text_unit_ids": r.text_unit_ids,
            "rank": r.rank,
            "weight": r.weight
        }
    }
    relationships_dict.append(relationship)


In [ ]:
from yfiles_jupyter_graphs import GraphWidget

w = GraphWidget()
w.directed = True
w.nodes = entities_dict
w.edges = relationships_dict
w.node_label_mapping = "title"
# map community to a color
def community_to_color(community):
    """Map a community to a color."""
    colors = [
        "crimson",
        "darkorange",
        "indigo",
        "cornflowerblue",
        "cyan",
        "teal",
        "green",
    ]
    return (
        colors[int(community) % len(colors)] if community is not None else "lightgray"
    )


def edge_to_source_community(edge):
    """Get the community of the source node of an edge."""
    source_node = next(
        (entry for entry in w.nodes if entry["properties"]["title"] == edge["start"]),
        None,
    )
    source_node_community = source_node["properties"]["community_ids"]
    return source_node_community if source_node_community is not None else None


w.node_color_mapping = lambda node: community_to_color(node["properties"]["community_ids"][0])
#w.edge_color_mapping = lambda edge: community_to_color(edge_to_source_community(edge))
# map size data to a reasonable factor
w.node_scale_factor_mapping = lambda node: 0.5 + node["properties"]["rank"] * 1.5 / 20
# use weight for edge thickness
w.edge_thickness_factor_mapping = "weight"
w.circular_layout()

In [6]:

display(w)

GraphWidget(layout=Layout(height='800px', width='100%'))